In [179]:
#load packages
import pandas as pd 
import numpy as np
import fuzzymatcher
pd.set_option('display.max_columns', 999)

In [180]:
#load data and convert dates to datetime
rw = pd.read_csv("../Datasets/RW_tail_numbers.csv")
rw.head()

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,arrival,duration,TailNumber,NAME
0,N101AE,20-Feb-2021,B06,Indy South Greenwood (KHFY),"Indy South Greenwood (Indianapolis, IN) - KHFY",Sanders Gyroport (IN88),"Sanders Gyroport (Cloverdale, IN) - IN88",03:56PM EST,04:15PM EST (?),0:18,N101AE,AIR EVAC EMS INC ...
1,N101AE,20-Feb-2021,B06,Owens Fld (II29),"Owens Fld (Greencastle, IN) - II29",Indy South Greenwood (KHFY),"Indy South Greenwood (Indianapolis, IN) - KHFY",03:01PM EST,03:21PM EST,0:20,N101AE,AIR EVAC EMS INC ...
2,N101AE,19-Feb-2021,B06,Eagle Creek Airpark (KEYE),"Eagle Creek Airpark (Indianapolis, IN) - KEYE","Near Greencastle, IN",L 39.61981 -86.70046,02:38PM EST,Last seen 02:50PM EST,0:11,N101AE,AIR EVAC EMS INC ...
3,N101AE,19-Feb-2021,B06,"Near Terre Haute, IN",L 39.24110 -87.12544,Eagle Creek Airpark (KEYE),"Eagle Creek Airpark (Indianapolis, IN) - KEYE",First seen 12:30PM EST,02:11PM EST,1:40,N101AE,AIR EVAC EMS INC ...
4,N101AE,16-Feb-2021,B06,"Near Terre Haute, IN",L 39.26091 -87.10094,"Near Indianapolis, IN",L 39.71667 -86.38333,First seen 01:51PM EST,Last seen 03:48PM EST,1:56,N101AE,AIR EVAC EMS INC ...


In [181]:
rw.shape

(116292, 12)

In [182]:
# Remove useless words
rw = rw.replace(regex={'Near ':'', 'First seen ':'', 'Last seen ':'', 'En Route':None, 'Unknown':None, 'Diverted':None})

In [183]:
# check for duplicates
rw.duplicated().sum()

0

In [184]:
# check for null values
rw.isna().sum()

tail_number                0
date                       0
aircraft                6114
origin                     1
origin_location            1
destination               83
destination_location      83
departure                  0
arrival                  194
duration                 232
TailNumber                 0
NAME                       0
dtype: int64

In [185]:
# split origin_Latitude and origin_Longitude
rw['origin_Latitude'] = rw[rw.origin_location.str.startswith('L ', na=False)].origin_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[0]
rw['origin_Longitude'] = rw[rw.origin_location.str.startswith('L ', na=False)].origin_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[1]

In [186]:
# split destination_Latitude and destination_Longitude
rw['destination_Latitude'] = rw[rw.destination_location.str.startswith('L ', na=False)].destination_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[0]
rw['destination_Longitude'] = rw[rw.destination_location.str.startswith('L ', na=False)].destination_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[1]

In [187]:
# convert to numeric
rw['origin_Latitude'] = pd.to_numeric(rw.origin_Latitude, errors='coerce')
rw['origin_Longitude'] = pd.to_numeric(rw.origin_Longitude, errors='coerce')

rw['destination_Latitude'] = pd.to_numeric(rw.destination_Latitude, errors='coerce')
rw['destination_Longitude'] = pd.to_numeric(rw.destination_Longitude, errors='coerce')

In [188]:
# split ICAO_code
rw['ICAO_code_origin'] = rw[rw.origin_Latitude.isna()].origin_location.str.rsplit(" - ", n = 1, expand = True)[1]
rw['ICAO_code_destination'] = rw[rw.destination_Latitude.isna()].destination_location.str.rsplit(" - ", n = 1, expand = True)[1]

In [189]:
rw.head()

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,arrival,duration,TailNumber,NAME,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,ICAO_code_destination
0,N101AE,20-Feb-2021,B06,Indy South Greenwood (KHFY),"Indy South Greenwood (Indianapolis, IN) - KHFY",Sanders Gyroport (IN88),"Sanders Gyroport (Cloverdale, IN) - IN88",03:56PM EST,04:15PM EST (?),0:18,N101AE,AIR EVAC EMS INC ...,NaN,NaN,NaN,NaN,KHFY,IN88
1,N101AE,20-Feb-2021,B06,Owens Fld (II29),"Owens Fld (Greencastle, IN) - II29",Indy South Greenwood (KHFY),"Indy South Greenwood (Indianapolis, IN) - KHFY",03:01PM EST,03:21PM EST,0:20,N101AE,AIR EVAC EMS INC ...,NaN,NaN,NaN,NaN,II29,KHFY
2,N101AE,19-Feb-2021,B06,Eagle Creek Airpark (KEYE),"Eagle Creek Airpark (Indianapolis, IN) - KEYE","Greencastle, IN",L 39.61981 -86.70046,02:38PM EST,02:50PM EST,0:11,N101AE,AIR EVAC EMS INC ...,NaN,NaN,39.61981,-86.70046,KEYE,NaN
3,N101AE,19-Feb-2021,B06,"Terre Haute, IN",L 39.24110 -87.12544,Eagle Creek Airpark (KEYE),"Eagle Creek Airpark (Indianapolis, IN) - KEYE",12:30PM EST,02:11PM EST,1:40,N101AE,AIR EVAC EMS INC ...,39.24110,-87.12544,NaN,NaN,NaN,KEYE
4,N101AE,16-Feb-2021,B06,"Terre Haute, IN",L 39.26091 -87.10094,"Indianapolis, IN",L 39.71667 -86.38333,01:51PM EST,03:48PM EST,1:56,N101AE,AIR EVAC EMS INC ...,39.26091,-87.10094,39.71667,-86.38333,NaN,NaN


In [190]:
# load airport code data
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")
airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [191]:
# check duplicates
airport_codes.duplicated().sum()

0

In [192]:
# split coordinates and convert to numeric
airport_codes["Latitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[1])
airport_codes["Longitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[0])

In [193]:
# merge origin airport information
rw = rw.merge(airport_codes[['ident', 'iata_code', 'Latitude', 'Longitude']].add_suffix('_origin'),
         how='left', left_on='ICAO_code_origin', right_on='ident_origin')

In [194]:
# merge destination airport information
rw = rw.merge(airport_codes[['ident', 'iata_code', 'Latitude', 'Longitude']].add_suffix('_destination'),
         how='left', left_on='ICAO_code_destination', right_on='ident_destination')

In [195]:
# merge 2 columns
rw.origin_Latitude = rw.origin_Latitude.fillna(0) + rw.Latitude_origin.fillna(0)
rw.origin_Longitude = rw.origin_Longitude.fillna(0) + rw.Longitude_origin.fillna(0)

rw.destination_Latitude = rw.destination_Latitude.fillna(0) + rw.Latitude_destination.fillna(0)
rw.destination_Longitude = rw.destination_Longitude.fillna(0) + rw.Longitude_destination.fillna(0)

# drop duplicated columns
rw = rw.drop(['ident_origin', 'ident_destination', 'Latitude_origin', 'Longitude_origin', 'Latitude_destination', 'Longitude_destination'], axis=1)

In [196]:
# load airport data
airports = pd.read_csv("..//Datasets/Airports/BTS_Airports_LAT_LON.csv")

In [197]:
airports.head()

,AIRPORT_CODE,AIRPORT_NAME,CITY_NAME,COUNTRY_NAME,STATE_NAME,STATE_CODE,LATITUDE,LONGITUDE
0,01A,Afognak Lake Airport,Afognak Lake - AK,United States,Alaska,AK,58.109444,-152.906667
1,03A,Bear Creek Mining Strip,Granite Mountain - AK,United States,Alaska,AK,65.548056,-161.071667
2,04A,Lik Mining Camp,Lik - AK,United States,Alaska,AK,68.083333,-163.166667
3,05A,Little Squaw Airport,Little Squaw - AK,United States,Alaska,AK,67.570000,-148.183889
4,06A,Kizhuyak Bay,Kizhuyak - AK,United States,Alaska,AK,57.745278,-152.882778


In [198]:
# drop other countries
airports = airports.query("COUNTRY_NAME == 'United States'")

In [199]:
# replace abbreviations to full name for the best matching
rw = rw.replace(regex={r'Muni': 'Municipal', r'Rgnl':'Regional', r'Intl':'International', r"Int'l":'International',
                 r'Trml':'Terminal', r'Fld':'Field'})

In [200]:
# split states fot the best matching
rw['origin_state'] = rw.origin.str.split(', ', n=1, expand=True)[1]
rw.loc[rw.origin_state.isna(), 'origin_state'] = rw.origin_location.str.split(', ', n=1, expand=True)[1].str.split(')', n=1, expand=True)[0]

rw['destination_state'] = rw.destination.str.split(', ', n=1, expand=True)[1]
rw.loc[rw.destination_state.isna(), 'destination_state'] = rw.destination_location.str.split(', ', n=1, expand=True)[1].str.split(')', n=1, expand=True)[0]

In [201]:
# fuzzy merge on locations with origin airports
rw = fuzzymatcher.fuzzy_left_join(rw, airports.add_suffix('_origin'), left_on = ['origin_state', 'origin_Latitude', 'origin_Longitude', 'origin', 'origin_location'], 
                                  right_on = ['STATE_CODE_origin', 'LATITUDE_origin', 'LONGITUDE_origin','AIRPORT_NAME_origin', 'CITY_NAME_origin'])\
                 .iloc[:,3:].reset_index(drop=True)

In [215]:
# drop missmatch in origin_state and iata_code_origin
rw.loc[(rw.origin_state != rw.STATE_CODE_origin) & (rw.iata_code_origin != rw.AIRPORT_CODE_origin), 
   ['AIRPORT_CODE_origin','AIRPORT_NAME_origin', 'CITY_NAME_origin', 'COUNTRY_NAME_origin', 
    'STATE_NAME_origin', 'STATE_CODE_origin', 'LATITUDE_origin', 'LONGITUDE_origin']] = None

In [203]:
# replace null values in iata_code_origin
rw.loc[rw.iata_code_origin.isna(),'iata_code_origin']  = rw.loc[rw.iata_code_origin.isna(), 'AIRPORT_CODE_origin']

In [204]:
# fuzzy merge on locations with destination airports
rw = fuzzymatcher.fuzzy_left_join(rw, airports.add_suffix('_destination'), left_on = ['destination_state', 'destination_Latitude', 'destination_Longitude', 'destination', 'destination_location'], 
                                  right_on = ['STATE_CODE_destination', 'LATITUDE_destination', 'LONGITUDE_destination','AIRPORT_NAME_destination', 'CITY_NAME_destination']).iloc[:,3:].reset_index(drop=True)

In [216]:
# drop missmatch in destination_state and iata_code_destination
rw.loc[(rw.destination_state != rw.STATE_CODE_destination) & (rw.iata_code_destination != rw.AIRPORT_CODE_destination), 
   ['AIRPORT_CODE_destination', 'AIRPORT_NAME_destination', 'CITY_NAME_destination', 'COUNTRY_NAME_destination', 'STATE_NAME_destination', 
    'STATE_CODE_destination', 'LATITUDE_destination', 'LONGITUDE_destination']] = None

In [206]:
# replace null values in iata_code_destination
rw.loc[rw.iata_code_destination.isna(),'iata_code_destination']  = rw.loc[rw.iata_code_destination.isna(), 'AIRPORT_CODE_destination']

In [213]:
# drop information with null location in origin
rw.loc[(rw.origin_Latitude == 0) & (rw.origin_Longitude == 0),
   ['AIRPORT_CODE_origin', 'AIRPORT_NAME_origin', 'CITY_NAME_origin', 'COUNTRY_NAME_origin', 'STATE_NAME_origin', 'STATE_CODE_origin', 'LATITUDE_origin', 'LONGITUDE_origin']] = None

In [217]:
# drop information with null location in destination
rw.loc[(rw.destination_Latitude == 0) & (rw.destination_Longitude == 0),
 ['AIRPORT_CODE_destination', 'AIRPORT_NAME_destination', 'CITY_NAME_destination', 'COUNTRY_NAME_destination', 
  'STATE_NAME_destination', 'STATE_CODE_destination', 'LATITUDE_destination', 'LONGITUDE_destination']] = None

In [177]:
# drop duplicated rows
rw = rw.drop(['destination_state', 'AIRPORT_CODE_destination', 'LATITUDE_destination', 'LONGITUDE_destination'], axis=1)
rw = rw.drop(['origin_state', 'AIRPORT_CODE_origin', 'LATITUDE_origin', 'LONGITUDE_origin'], axis=1)

In [221]:
# complete ICAO codes
rw.loc[rw.ICAO_code_origin.isna(), 'ICAO_code_origin'] = rw.merge(airport_codes[['iata_code','ident']], how='left', left_on='iata_code_origin', right_on='iata_code').ident
rw.loc[rw.ICAO_code_destination.isna(), 'ICAO_code_destination'] = rw.merge(airport_codes[['iata_code','ident']], how='left', left_on='iata_code_destination', right_on='iata_code').ident

In [226]:
rw = rw.drop('TailNumber', axis=1)

In [227]:
rw.to_csv('..//Datasets/RW_with_airports.csv', index=False)

In [228]:
rw

,tail_number,date,aircraft,origin,origin_location,destination,destination_location,departure,arrival,duration,NAME,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,ICAO_code_origin,ICAO_code_destination,iata_code_origin,iata_code_destination,origin_state,destination_state,AIRPORT_CODE_origin,AIRPORT_NAME_origin,CITY_NAME_origin,COUNTRY_NAME_origin,STATE_NAME_origin,STATE_CODE_origin,LATITUDE_origin,LONGITUDE_origin,AIRPORT_CODE_destination,AIRPORT_NAME_destination,CITY_NAME_destination,COUNTRY_NAME_destination,STATE_NAME_destination,STATE_CODE_destination,LATITUDE_destination,LONGITUDE_destination
0,N101AE,20-Feb-2021,B06,Indy South Greenwood (KHFY),"Indy South Greenwood (Indianapolis, IN) - KHFY",Sanders Gyroport (IN88),"Sanders Gyroport (Cloverdale, IN) - IN88",03:56PM EST,04:15PM EST (?),0:18,AIR EVAC EMS INC ...,39.628399,-86.087898,39.513401,-86.763901,KHFY,IN88,IN7,IN7,IN,IN,IN7,Indy South Greenwood,Indianapolis - IN,United States,Indiana,IN,39.627500,-86.088056,IN7,Greenwood Municipal,Indianapolis - IN,United States,Indiana,IN,39.627500,-86.088056
1,N101AE,20-Feb-2021,B06,Owens Field (II29),"Owens Field (Greencastle, IN) - II29",Indy South Greenwood (KHFY),"Indy South Greenwood (Indianapolis, IN) - KHFY",03:01PM EST,03:21PM EST,0:20,AIR EVAC EMS INC ...,39.610901,-86.756104,39.628399,-86.087898,II29,KHFY,IN7,IN7,IN,IN,IN7,Greenwood Municipal,Indianapolis - IN,United States,Indiana,IN,39.627500,-86.088056,IN7,Indy South Greenwood,Indianapolis - IN,United States,Indiana,IN,39.627500,-86.088056
2,N101AE,19-Feb-2021,B06,Eagle Creek Airpark (KEYE),"Eagle Creek Airpark (Indianapolis, IN) - KEYE","Greencastle, IN",L 39.61981 -86.70046,02:38PM EST,02:50PM EST,0:11,AIR EVAC EMS INC ...,39.830700,-86.294403,39.619810,-86.700460,KEYE,NaN,IN4,IN7,IN,IN,IN4,Eagle Creek Airpark,Indianapolis - IN,United States,Indiana,IN,39.830833,-86.294444,IN7,Greenwood Municipal,Indianapolis - IN,United States,Indiana,IN,39.627500,-86.088056
3,N101AE,19-Feb-2021,B06,"Terre Haute, IN",L 39.24110 -87.12544,Eagle Creek Airpark (KEYE),"Eagle Creek Airpark (Indianapolis, IN) - KEYE",12:30PM EST,02:11PM EST,1:40,AIR EVAC EMS INC ...,39.241100,-87.125440,39.830700,-86.294403,KHUF,KEYE,HUF,IN4,IN,IN,HUF,Terre Haute Regional,Terre Haute - IN,United States,Indiana,IN,39.450556,-87.306944,IN4,Eagle Creek Airpark,Indianapolis - IN,United States,Indiana,IN,39.830833,-86.294444
4,N101AE,16-Feb-2021,B06,"Terre Haute, IN",L 39.26091 -87.10094,"Indianapolis, IN",L 39.71667 -86.38333,01:51PM EST,03:48PM EST,1:56,AIR EVAC EMS INC ...,39.260910,-87.100940,39.716670,-86.383330,KHUF,KIND,HUF,IND,IN,IN,HUF,Terre Haute Regional,Terre Haute - IN,United States,Indiana,IN,39.450556,-87.306944,IND,Indianapolis International,Indianapolis - IN,United States,Indiana,IN,39.725556,-86.283889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116287,N135MH,30-Nov-2020,EC35,Maryland (2W5),"Maryland (Indian Head, MD) - 2W5",Maryland (2W5),"Maryland (Indian Head, MD) - 2W5",09:46PM EST,10:39PM EST,0:53,WASHINGTON HOSPITAL CENTER ...,0.000000,0.000000,0.000000,0.000000,2W5,2W5,MD2,MD2,MD,MD,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN
116288,N135MH,30-Nov-2020,EC35,Maryland (2W5),"Maryland (Indian Head, MD) - 2W5",Maryland (2W5),"Maryland (Indian Head, MD) - 2W5",05:02PM EST,05:02PM EST (?),0:00,WASHINGTON HOSPITAL CENTER ...,0.000000,0.000000,0.000000,0.000000,2W5,2W5,MD2,MD2,MD,MD,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN
116289,N135MH,30-Nov-2020,EC35,Maryland (2W5),"Maryland (Indian Head, MD) - 2W5",Reagan National (KDCA),"Reagan National (Washington, DC) - KDCA",02:43PM EST,04:28PM EST,1:45,WASHINGTON HOSPITAL CENTER ...,0.000000,0.000000,38.852100,-77.037697,2W5,KDCA,MD2,DCA,MD,DC,None,None,None,None,None,None,NaN,NaN,DCA,Ronald Reagan Washington National,Washington - DC,United States,Virgi